This notebook provides code that processes the [SGCN/SWAP repository in ScienceBase](https://www.sciencebase.gov/catalog/item/56d720ece4b015c306f442d5) into the SGCN database in the GC2 instance we are experimenting with. It works through all of the items in the repo, grabs the source data file, checks the source data file record count against the current database, wipes the current database for that state/year if there's a problem, inserts all records from the source file, and verifies that the new record count matches.

### Updates (5/30/2017)
* I fixed a dumb problem where "taxonomy category" was not getting picked up and processed as "taxonomic_group".
* I added in a process to check for a list of states to reprocess explicitly in addition to those whose record count does not match source data. This enabled me to clean up an issue where some of the states didn't get their taxonomic group processed properly.
* I changed the method of detecting the file to operate on and set it up to look first for an update file. If no processable file is found, the script breaks out of the ScienceBase source item and moves on to the next one.

I use jupyter nbconvert to output this notebook to a python script and run it in its own environment to fully process the repository.

In [1]:
import requests,io
from IPython.display import display
import pandas as pd

# BIS package is a private set of config data available to BCB personnel
from bis import gc2

In [2]:
def getCurrentRecordCount(sgcn_state,sgcn_year):
    r = requests.get(gc2.baseURLs["sqlAPI_DataDistillery_BCB"]+"?q=SELECT COUNT(*) AS sumstateyear FROM sgcn.sgcn WHERE sgcn_year="+str(sgcn_year)+" AND sgcn_state='"+sgcn_state+"'").json()
    return r["features"][0]["properties"]["sumstateyear"]

In [3]:
def clearStateYear(sgcn_state,sgcn_year):
    return requests.get(gc2.baseURLs["sqlAPI_DataDistillery_BCB"]+"?key="+gc2.gc2Keys["DataDistillery_BCB"]+"&q=DELETE FROM sgcn.sgcn WHERE sgcn_year="+str(sgcn_year)+" AND sgcn_state='"+sgcn_state+"'").json()

In [4]:
def insertSGCNData(record):
    q = "INSERT INTO sgcn.sgcn (sourceid,sourcefilename,sourcefileurl,sgcn_state,sgcn_year,scientificname_submitted,commonname_submitted,taxonomicgroup_submitted,firstyear) \
        VALUES \
        ('"+record["sourceid"]+"','"+record["sourcefilename"]+"','"+record["sourcefileurl"]+"','"+record["sgcn_state"]+"',"+str(record["sgcn_year"])+",'"+record["scientificname_submitted"]+"','"+record["commonname_submitted"]+"','"+record["taxonomicgroup_submitted"]+"',"+str(record["firstyear"])+")"
    return requests.get(gc2.baseURLs["sqlAPI_DataDistillery_BCB"]+"?key="+gc2.gc2Keys["DataDistillery_BCB"]+"&q="+q).json()

In [5]:
def stringCleaning(text):
    import re

    # Specify replacements
    replacements = {}
    replacements["'"] = "''"
    replacements["--"] = ""
    replacements["&"] = "and"
    replacements['"'] = "''"
    replacements[";"] = ","
    replacements["#"] = "no."
    
    # Compile the expressions
    regex = re.compile("(%s)" % "|".join(map(re.escape, replacements.keys())))

    # Strip the text
    text = text.strip()

    # Process replacements
    return regex.sub(lambda mo: replacements[mo.string[mo.start():mo.end()]], text)

In [7]:
# Query ScienceBase for all SGCN source items
sbQ = "https://www.sciencebase.gov/catalog/items?parentId=56d720ece4b015c306f442d5&format=json&fields=files,tags,dates&max=100"
sbR = requests.get(sbQ).json()

In [8]:
# To keep from overloading the Jupyter Notebook version of this script, I added a process to run only one state/year combination all the way through. This can be removed in the standalong py version.
numberTests = 0

# Set a list with "state/year" values to reprocess explicitly
reprocessList = ["Arizona/2015"]

# Set a value of the total records in all files to report on at the end of processing ScienceBase Items
totalRecordsInFiles = 0

# Loop through the repository items and sync data to SGCN database
for item in sbR["items"]:
    # Create an iterative structure to contain processed attributes for a given state and year
    thisItem = {}
    
    # Set the source id as the ScienceBase Item URI/URL to reference from all final records
    thisItem["sourceid"] = item["link"]["url"]
    
    # Extract the state name from the place tag
    # NOTE: This is brittle and needs more work. An item could have many tags, and I just shortcutted to this point based on how Abby has managed these items so far.
    try:
        thisItem["sgcn_state"] = item['tags'][0]['name']
    except:
        display (thisItem)
        break
    
    # Extract the year for this item from the dates collection based on the "Collected" date type
    # Break after we find the date collected that we want to use
    for date in item["dates"]:
        if date["type"] == "Collected":
            thisItem["sgcn_year"] = date["dateString"]
            break
    
    # Retrieve the current record count in the SGCN database for the state and year
    thisItem["startingrecordcount"] = getCurrentRecordCount(thisItem["sgcn_state"],thisItem["sgcn_year"])
    
    # Extract the file we need to process from the files structure
    # First look for an update file and process that one. Otherwise get the original file.
    updateFile = next((file for file in item["files"] if file["title"] == "Process ready version of updated file"), None)
    if updateFile is not None:
        processFile = updateFile
    else:
        newFile = next((file for file in item["files"] if file["title"] == "Process ready version of original file"), None)
        if newFile is not None:
            processFile = newFile
        else:
            processFile = None

    if processFile is not None:
        thisItem["sourcefilename"] = processFile["name"]
        thisItem["sourcefileurl"] = processFile["url"]
    else:
        print ("Problem getting file prepared: "+thisItem["sourceid"])
        break
        
    
    # Retrieve the file into a dataframe for processing
    # The read_table method with explicit tab separator seems to be pretty reliable and robust directly from ScienceBase file URLs, but this may have to be reexamined in future if it fails
    stateData = pd.read_table(thisItem["sourcefileurl"],sep='\t')
    totalRecordsThisFile = len(stateData.index)
    totalRecordsInFiles = totalRecordsInFiles + totalRecordsThisFile

    # Check the total columns in the source data against the starting record count from the database
    # Also check to see if the state/year are explicitly in a list to reprocess for this run
    if thisItem["startingrecordcount"] != totalRecordsThisFile or thisItem["sgcn_state"]+"/"+str(thisItem["sgcn_year"]) in reprocessList:
        # If the number of source records does not match the current database, clear out the database for reprocessing
        print (clearStateYear(thisItem["sgcn_state"],thisItem["sgcn_year"]))
        print ("Cleared data for: "+thisItem["sgcn_state"]+" - "+str(thisItem["sgcn_year"]))
    
        # Store the source record count for reference
        thisItem["sourcerecordcount"] = len(stateData.index)
    
        # Set column names to lower case to deal with pesky human problems
        stateData.columns = map(str.lower, stateData.columns)
        
        thisRecord = {}
    
        # Loop the dataframe, fix text values, and load to SGCN database
        # NaN values from the dataframe are nulls, show up as float type, and need to get set to a blank string
        for index, row in stateData.iterrows():
            if type(row['scientific name']) is float:
                thisRecord["scientificname_submitted"] = ""
            else:
                thisRecord["scientificname_submitted"] = stringCleaning(row['scientific name'])

            if type(row['common name']) is float:
                thisRecord["commonname_submitted"] = ""
            else:
                thisRecord["commonname_submitted"] = stringCleaning(row['common name'])

            thisRecord["taxonomicgroup_submitted"] = ""
            if 'taxonomy group' in stateData.columns:
                thisRecord["taxonomicgroup_submitted"] = stringCleaning(row['taxonomy group'])
            elif 'taxonomic category' in stateData.columns:
                thisRecord["taxonomicgroup_submitted"] = stringCleaning(row['taxonomic category'])

            thisRecord["firstyear"] = False
            if '2005 swap' in stateData.columns:
                if row['2005 swap'] in ["N","n","No","no"]:
                    thisRecord["firstyear"] = True
            
            # Add in repository item metadata
            thisRecord["sourceid"] = thisItem["sourceid"]
            thisRecord["sourcefilename"] = thisItem["sourcefilename"]
            thisRecord["sourcefileurl"] = thisItem["sourcefileurl"]
            thisRecord["sgcn_state"] = thisItem["sgcn_state"]
            thisRecord["sgcn_year"] = thisItem["sgcn_year"]
            
            # Insert the record
            print(insertSGCNData(thisRecord))
        
        # Check total record count after inserting new data to make sure the numbers line up
        if thisItem["sourcerecordcount"] != getCurrentRecordCount(thisItem["sgcn_state"],thisItem["sgcn_year"]):
            print ("Something went wrong with "+thisItem["sgcn_state"],thisItem["sgcn_year"],thisItem["sourceid"])
        else:
            print (thisItem["sgcn_state"],thisItem["sgcn_year"])
            print ("Source record count: "+str(thisItem["sourcerecordcount"]))
            print ("Database record count: "+str(getCurrentRecordCount(thisItem["sgcn_state"],thisItem["sgcn_year"])))
        
        numberTests = numberTests + 1
    else:
        print ("Record Numbers Matched: "+thisItem["sgcn_state"]+" - "+str(thisItem["sgcn_year"]))
    
    if numberTests > 0:
        break

print ("Total records in all files processed: "+str(totalRecordsInFiles))
    

{'affected_rows': 530, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.118}
Cleared data for: Arizona - 2015
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.073}
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.066}
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.068}
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.069}
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.066}
{'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.066}
{'affected_rows': 1, 'succe